In [19]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

pd.set_option('display.max_colwidth', -1)
np.set_printoptions(threshold=np.nan)

In [20]:
# import data
loans = pd.read_csv('lending-club-data.csv',dtype = {'desc':np.str,'next_pymnt_d':np.str})

loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans.pop('bad_loans')

features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

loans = loans[features + [target]]
# split into training and validation

import json 
with open('module-6-assignment-validation-idx.json') as valid_index:
    valid_index = json.load(valid_index)
with open('module-6-assignment-train-idx.json') as train_index:
    train_index = json.load(train_index)

In [21]:
categorical_variables = []
for feat_name,feat_type in zip(list(loans.columns),list(loans.dtypes)):
    if feat_type == "object":
        categorical_variables.append(feat_name)
        
for feature in categorical_variables:
    globals()['df_'+feature] = pd.get_dummies(loans[feature])
    globals()['df_'+feature].columns = [feature+'_'+str(col) for col in  \
                                        globals()['df_'+feature].columns]
    loans.pop(feature)
    loans = pd.concat([loans,globals()['df_'+feature]],axis = 1)

In [22]:
# Split into training and validation

train_data = loans.iloc[train_index]
validation_data = loans.iloc[valid_index]

Early stopping condition 1: Maximum depth

Recall that we already implemented the maximum depth stopping condition in the previous assignment. In this assignment, we will experiment with this condition a bit more and also write code to implement the 2nd and 3rd early stopping conditions.

We will be reusing code from the previous assignment and then building upon this. We will alert you when you reach a function that was part of the previous assignment so that you can simply copy and past your previous code.

Early stopping condition 2: Minimum node size

The function reached_minimum_node_size takes 2 arguments:

The data (from a node)
The minimum number of data points that a node is allowed to split on, min_node_size.
 This function simply calculates whether the number of data points at a given node is less than or equal to the specified minimum node size. This function will be used to detect this early stopping condition in the decision_tree_create function. Your code should be analogous to

In [23]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    return len(data) == min_node_size

Early stopping condition 3: Minimum gain in error reduction

The function error_reduction takes 2 arguments:

The error before a split, error_before_split.
The error after a split, error_after_split.

This function computes the gain in error reduction, i.e., the difference between the error before the split and that after the split. This function will be used to detect this early stopping condition in the decision_tree_create function. Your code should be analogous to

In [24]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    return error_before_split-error_after_split

In [25]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (safe loans)
    count_of_positives = list(labels_in_node).count(1)
    # Count the number of -1's (risky loans)
    count_of_negatives = list(labels_in_node).count(-1)         
    # Return the number of mistakes that the majority classifier makes.
    if count_of_positives> count_of_negatives:
        return count_of_negatives
    else:
        return count_of_positives

In [26]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split = data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target])           

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = (left_mistakes + right_mistakes)/ num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_error = error
            best_feature = feature
        
    
    return best_feature # Return the best feature we found

In [27]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': None,
            'prediction': None}   ## YOUR CODE HERE 
   
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = num_ones
        leaf['is_leaf'] = True## YOUR CODE HERE
    else:
        leaf['prediction'] = num_minus_ones
        leaf['is_leaf'] = True ## YOUR CODE HERE        

    # Return the leaf node
    return leaf 

In [28]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print ("Stopping condition 1 reached. All data points have the same target value.")               
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == []:
        print ("Stopping condition 2 reached. No remaining features.")              
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print ("Early stopping condition 1 reached. Reached maximum depth.")
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if  float(len(data)) == 0.0:         ## YOUR CODE HERE 
        print ("Early stopping condition 2 reached. Reached minimum node size.")
        return create_leaf(target_values)   ## YOUR CODE HERE
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes =  intermediate_node_num_mistakes(left_split[target])   ## YOUR CODE HERE
    right_mistakes =  intermediate_node_num_mistakes(right_split[target])  ## YOUR CODE HERE
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if  error_reduction(error_before_split, error_after_split) <= min_error_reduction:      ## YOUR CODE HERE
        print ("Early stopping condition 3 reached. Minimum error reduction.")
        return create_leaf(target_values) ## YOUR CODE HERE 
    
    
    remaining_features.remove(splitting_feature)
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth) 

    
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [43]:
features = list(train_data.columns)
features.remove('safe_loans')
my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length_n/a. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Split on feature emp_length_< 1 year. (85, 11)
--------------------------------------------------------------------
Subtree, depth = 4 (85 data points).
Early stopping condition 3 reached. Minimum error reduction.
------------------------------------------------------------

In [45]:
my_decision_tree_old = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,          
                                            min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

Making predictions

Recall that in the previous assignment you implemented a function classify to classify a new point x using a given tree. We will need that function here. Use your code from the previous assignment. It should be analogous to

In [52]:
def classify(tree, x, annotate = False):
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [70]:
print (validation_data.iloc[0])
print ('Predicted class: %s ' % classify(my_decision_tree_new, validation_data.iloc[0],True))

safe_loans                -1.0
grade_A                    0.0
grade_B                    0.0
grade_C                    0.0
grade_D                    1.0
grade_E                    0.0
grade_F                    0.0
grade_G                    0.0
term_ 36 months            0.0
term_ 60 months            1.0
home_ownership_MORTGAGE    0.0
home_ownership_OTHER       0.0
home_ownership_OWN         0.0
home_ownership_RENT        1.0
emp_length_1 year          0.0
emp_length_10+ years       0.0
emp_length_2 years         1.0
emp_length_3 years         0.0
emp_length_4 years         0.0
emp_length_5 years         0.0
emp_length_6 years         0.0
emp_length_7 years         0.0
emp_length_8 years         0.0
emp_length_9 years         0.0
emp_length_< 1 year        0.0
emp_length_n/a             0.0
Name: 24, dtype: float64
Split on term_ 36 months = 0.0
Split on grade_A = 0.0
At leaf, predicting 5963
Predicted class: 5963 
